In [ ]:
import pandas as pd

In [ ]:
import numpy as np
from datetime import datetime
from datetime import datetime, timedelta


In [ ]:
df_glucose = pd.read_csv('path'', sep=',')


In [ ]:
df_glucose = df_glucose.sort_values(by=['id', 'Time'], ascending=True)

In [ ]:
def give_fasting_state(row):
    
    time = row["Time"]
    
    time = datetime.strptime(time, "%Y-%m-%d %H:%M:%S")
    time =  time.strftime('%H:%M:%S')
    time = datetime.strptime(time,'%H:%M:%S').time()
    
    
    
    breakfast_begin = datetime.strptime("08:00:00", "%H:%M:%S").time()
    breakfast_end = datetime.strptime("10:00:00", "%H:%M:%S").time()

    lunch_begin = datetime.strptime("12:00:00", "%H:%M:%S").time()
    lunch_end = datetime.strptime("14:00:00", "%H:%M:%S").time()
    
    dinner_begin = datetime.strptime("18:00:00", "%H:%M:%S").time()
    dinner_end = datetime.strptime("20:00:00", "%H:%M:%S").time()
    
    if (time > breakfast_begin and time < breakfast_end or time > lunch_begin and time < lunch_end or time > dinner_begin and time < dinner_end ):
        
        fasting_state  = "non-fasting"
    
    else:
        
        fasting_state  = "fasting"
        
    return fasting_state

In [ ]:
df_glucose["fasting_state"] = df_glucose.apply(give_fasting_state, axis=1)

In [ ]:
df_glucose.to_csv("simglucose_data.csv", sep='\t')



# Adding Fasting states to overnight fasting

In [ ]:
df_glucose_overnight = pd.read_csv('path', index_col=0, sep='\t')


In [ ]:
df_glucose_overnight

In [ ]:
df_glucose_overnight["time"] = df_glucose_overnight["time"].replace(['2020-09-02 22:04:59.995000+00:00'], '2020-09-02 22:04:00+00:00')

df_glucose_overnight = df_glucose_overnight.sort_values(by=['id', 'time'], ascending=True)

In [ ]:
overnight_overlap = datetime.strptime(("23:00:00"), "%H:%M:%S")
overnight_overlap =  overnight_overlap.strftime('%H:%M:%S')
overnight_overlap = datetime.strptime(overnight_overlap,'%H:%M:%S').time()


for index_1, row_df in df_glucose_overnight.iterrows():
        
    
    for index_2, row_df_fasting in df_fasting_times_all.iterrows():
        
        
        ###### from merged df ######
        try:
            time_df_merged = datetime.strptime(str(row_df["time"]), "%Y-%m-%d %H:%M:%S%z")
            date_df_merged = time_df_merged.strftime('%Y-%m-%d')
            date_df_merged = datetime.strptime(date_df_merged,'%Y-%m-%d')

            min_df_merged =  time_df_merged.strftime('%H:%M:%S')
            min_df_merged = datetime.strptime(min_df_merged,'%H:%M:%S').time()
        except:
            time_df_merged = datetime.strptime(str(row_df["time"]), "%Y-%m-%d %H:%M:%S+%z")
            date_df_merged = time_df_merged.strftime('%Y-%m-%d')
            date_df_merged = datetime.strptime(date_df_merged,'%Y-%m-%d')

            min_df_merged =  time_df_merged.strftime('%H:%M:%S')
            min_df_merged = datetime.strptime(min_df_merged,'%H:%M:%S').time()        
        ###### from fasting df #######
        # day
        
        time_fasting = datetime.strptime(str(row_df_fasting["time"]), "%Y-%m-%d %H:%M:%S")
        date_fasting = time_fasting.strftime('%Y-%m-%d')
        date_fasting = datetime.strptime(date_fasting,'%Y-%m-%d')
        
        # minutes

        begin_non_fasting = datetime.strptime(str(row_df_fasting["Beginn erste Mahlzeit"]), "%H:%M:%S")
        begin_non_fasting =  begin_non_fasting.strftime('%H:%M:%S')
        begin_non_fasting = datetime.strptime(begin_non_fasting,'%H:%M:%S').time()


        if not row_df_fasting["Beginn letzte Mahlzeit"] == "nan":
            end_non_fasting = datetime.strptime(str(row_df_fasting["Beginn letzte Mahlzeit"]), "%H:%M:%S")
            end_non_fasting =  end_non_fasting.strftime('%H:%M:%S')
            end_non_fasting = datetime.strptime(end_non_fasting,'%H:%M:%S').time()
            


        
        if row_df["id"] == row_df_fasting["id"] and row_df["phase"] == row_df_fasting["phase"] and date_df_merged == date_fasting:
            
            if begin_non_fasting < end_non_fasting:
                # begin_non_fasting 10 Uhr
                # end_non_fasting  22 Uhr
                if min_df_merged >= begin_non_fasting and min_df_merged < end_non_fasting:
                    df_glucose_overnight.at[index_1,'label']  = "non-fasting" 
            else: 
                
                if min_df_merged >= begin_non_fasting and min_df_merged < overnight_overlap:
                    # nicht fastenzustand
                    df_glucose_overnight.at[index_1,'label']  = "non-fasting"                 




In [ ]:
df_glucose_overnight.to_csv(".csv", sep='\t')
